# Christmas Eve

In [6]:
import numpy as np

In [653]:
bugs = """#..##
##...
.#.#.
#####
####."""

In [242]:
def to_array(string):
    lines = string.split("\n")
    array = np.array([[l for l in t] for t in lines])
    np.place(array, array=='#', 1)
    np.place(array, array=='.', 0)
    np.place(array, array=='?', 0)
    return array.astype(int)

In [41]:
def n_adjacent(array):
    return \
    (np.concatenate((np.zeros_like(array[0:1]),
                array[:-1]), axis=0)) + \
    (np.concatenate((array[1:],
                     np.zeros_like(array[0:1])
                    ), axis=0)) + \
    (np.concatenate((np.zeros_like(array[:,0:1]),
                    array[:,:-1]), axis=1)) + \
    (np.concatenate((array[:,1:],
                     np.zeros_like(array[:,0:1])
                    ), axis=1))

In [98]:
def update(array):
    to_ret = array.copy()
    adj = n_adjacent(array)
    
    # Bug dies unless exactly 1 adj:
    np.place(to_ret, (array==1) & (adj!=1), 0)
    # Empty space infested if exactly 1 or 2 adj:
    np.place(to_ret, (array==0) & ((adj==1) | (adj==2)), 1)
    
    return to_ret

In [169]:
def score(array):
    return np.sum(np.power(2, np.arange(array.size)), where=(array.ravel()==1))

# Q1

In [654]:
bug_array = to_array(bugs)

scores = set()
while True:
    bug_array = update(bug_array)
    s = score(bug_array)
    if s in scores:
        print(s)
        break
    scores.add(s)

23967691


# Q2

In [210]:
from collections import defaultdict

In [578]:
def n_adjacent_recur(levels, level=0):
    array=levels[level].copy()
    
    adj = \
    (np.concatenate((np.zeros_like(array[0:1]) + levels[level-1][1,2],
                array[:-1]), axis=0)) + \
    (np.concatenate((array[1:],
                     np.zeros_like(array[0:1]) + levels[level-1][3,2]
                    ), axis=0)) + \
    (np.concatenate((np.zeros_like(array[:,0:1]) + levels[level-1][2,1],
                    array[:,:-1]), axis=1)) + \
    (np.concatenate((array[:,1:],
                     np.zeros_like(array[:,0:1])  + levels[level-1][2,3]
                    ), axis=1))
    
    adj[1,2] += levels[level+1][0].sum()
    adj[3,2] += levels[level+1][-1].sum()
    adj[2,1] += levels[level+1][:,0].sum()
    adj[2,3] += levels[level+1][:,-1].sum()
    
    return adj

In [635]:
def update_recur(levels, verbose=False):
    levels = levels.copy()
    
    # Create new levels:
    _ = levels[max(levels.keys())+1]
    _ = levels[min(levels.keys())-1]
    
    to_ret = levels.copy()
    
    keys = list(levels.keys()) # Will create new keys for new levels on the fly
    
    for level in keys:
        new = levels[level].copy()
        adj = n_adjacent_recur(levels, level=level)

        # Bug dies unless exactly 1 adj:
        np.place(new, (levels[level]==1) & (adj!=1), 0)
        # Empty space infested if exactly 1 or 2 adj:
        np.place(new, (levels[level]==0) & ((adj==1) | (adj==2)), 1)
        # Leave middle 0
        new[2,2] = 0
        to_ret[level] = new
        
        if verbose:
            print(level)
            print(levels[level])
            print(adj)
            print(new)

    return to_ret

In [657]:
bug_array = to_array(bugs)
bug_array

array([[1, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 1, 0, 1, 0],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0]])

In [658]:
levels = defaultdict(lambda : np.zeros_like(bug_array))
levels[0] = bug_array.copy()

for i in range(200):
    levels = update_recur(levels)

np.array([l for l in levels.values()]).sum()

2003